In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

# Historical target table
Hist_table=HTable.read(1394, with_units=False)
Hist_table


Years,Entries
IntegerAndSexagesimal,Historical
365 ;,"11s 29 ; 45,39,22"
730 ;,"11s 29 ; 31,18,44"
1095 ;,"11s 29 ; 16,58,06"
1461 ;,"00 ; 01,45,48"
1826 ;,"11s 29 ; 47,25,10"
2191 ;,"11s 29 ; 33,04,32"
2556 ;,"11s 29 ; 18,43,54"
2922 ;,"00 ; 03,31,36"
3287 ;,"11s 29 ; 49,10,58"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(34):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)

In [4]:
# mean motion parameter from the Parisian Alfonsine tables
daily_mean=Historical(Sexagesimal("0;59,8,19,37,19,13,56"),8)
REVO=Historical(Sexagesimal("6,0;0,0"),8)

for i in range(34):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=6):
        days=Hist_table[i][0]
        Test_recomp[i][1]=(daily_mean*days)%REVO
        with set_precision(tmode=TruncatureMode.ROUND, pmode=3):
            Test_recomp[i][1]=1*Test_recomp[i][1]
            Delta_recomp[i][1]=60*60*60*(Hist_table[i][1]-Test_recomp[i][1])


In [5]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(0.29411764705882354, nan)

In [6]:
B.astype(float).style.background_gradient(axis=None)

,Entries
Years,
365 ;,0.000000
730 ;,0.000000
1095 ;,0.000000
1461 ;,0.000000
1826 ;,0.000000
2191 ;,0.000000
2556 ;,0.000000
2922 ;,1.000000
3287 ;,0.000000


In [7]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Solar mean years.csv')
B.to_csv('Solar mean years.zip', index=False, compression=compression_opts)